# Jugando con la gramática

Temas a tratar:

* Contando sustantivos (plurales y singulares).
* Obtener el análisis de dependencia.
* División de frases en cláusulas.
* Extracción de trozos sustantivos.
* Entidades extractoras y relaciones.
* Extraer sujetos y objetos de la oración.
* Encontrar referencias.

## Contar sustantivos - sustantivos plurales y singulares

In [1]:
# Importando los módulos necesarios
import nltk
from nltk.stem import WordNetLemmatizer
import inflect

In [2]:
def tokenize_nltk(text):
    return nltk.tokenize.word_tokenize(text)

def pos_tag_nltk(text):
    words = tokenize_nltk(text)
    words_with_pos = nltk.pos_tag(words)
    return words_with_pos


In [3]:
# Lectura del texto en inglés
filename = "pv.txt"
file = open(filename, "r", encoding="utf-8")
text = file.read()
text = text.replace("\n", " ")

In [4]:
# Hacer parte del etiquetado del habla
words_with_pos = pos_tag_nltk(text)
print(words_with_pos)

[('A', 'DT'), ('photovoltaic', 'JJ'), ('array', 'NN'), ('(', '('), ('PVA', 'NNP'), (')', ')'), ('simulation', 'NN'), ('model', 'NN'), ('to', 'TO'), ('be', 'VB'), ('used', 'VBN'), ('in', 'IN'), ('Matlab-Simulink', 'NNP'), ('GUI', 'NNP'), ('environment', 'NN'), ('is', 'VBZ'), ('developed', 'VBN'), ('and', 'CC'), ('presented', 'VBN'), ('in', 'IN'), ('this', 'DT'), ('paper', 'NN'), ('.', '.'), ('The', 'DT'), ('model', 'NN'), ('is', 'VBZ'), ('developed', 'VBN'), ('using', 'VBG'), ('basic', 'JJ'), ('circuit', 'NN'), ('equations', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('photovoltaic', 'NN'), ('(', '('), ('PV', 'NNP'), (')', ')'), ('solar', 'VBP'), ('cells', 'NNS'), ('including', 'VBG'), ('the', 'DT'), ('effects', 'NNS'), ('of', 'IN'), ('solar', 'JJ'), ('irradiation', 'NN'), ('and', 'CC'), ('temperature', 'NN'), ('changes', 'NNS'), ('.', '.'), ('The', 'DT'), ('new', 'JJ'), ('model', 'NN'), ('was', 'VBD'), ('tested', 'VBN'), ('using', 'VBG'), ('a', 'DT'), ('directly', 'RB'), ('coupled', 'VBN'), 

In [5]:
# Definimos la función get_nouns, para filtrar los sustantivos:
def get_nouns(words_with_pos):
    noun_set = ["NN", "NNS"]
    nouns = [word for word in words_with_pos if  word[1] in noun_set]
    return nouns

In [6]:
# Ejecutamos la función anterior en la lista de palabras etiquetadas con POS:
nouns = get_nouns(words_with_pos)
print(nouns)

[('array', 'NN'), ('simulation', 'NN'), ('model', 'NN'), ('environment', 'NN'), ('paper', 'NN'), ('model', 'NN'), ('circuit', 'NN'), ('equations', 'NNS'), ('photovoltaic', 'NN'), ('cells', 'NNS'), ('effects', 'NNS'), ('irradiation', 'NN'), ('temperature', 'NN'), ('changes', 'NNS'), ('model', 'NN'), ('dc', 'NN'), ('load', 'NN'), ('load', 'NN'), ('inverter', 'NN'), ('validation', 'NN'), ('studies', 'NNS'), ('load', 'NN'), ('circuits', 'NNS'), ('results', 'NNS')]


Para determinar si un sustantivo es singular o plural, tenemos dos opciones. La primera opción es usar las etiquetas NLTK, donde NN indica un sustantivo singular y NNS indica un sustantivo plural. <br>

La siguiente función utiliza las etiquetas NLTK y devuelve True si el sustantivo de entrada es plural:

In [7]:
def is_plural_nltk(noun_info):
    pos = noun_info[1]
    if (pos == "NNS"):
        return True
    else:
        return False

In [8]:
is_plural_nltk(('equations', 'NNS'))

True

In [9]:
is_plural_nltk(('photovoltaic', 'NN'))

False

La otra opción es usar la clase WordNetLemmatizer en el paquete nltk.stem. La siguiente función devuelve True si el sustantivo es plural:

In [10]:
def is_plural_wn(noun):
    wnl = WordNetLemmatizer()
    lemma = wnl.lemmatize(noun, 'n')
    plural = True if noun is not lemma else False
    return plural

In [11]:
is_plural_wn('equations')

True

In [12]:
is_plural_wn('photovoltaic')

False

In [13]:
# La siguiente función cambiará un sustantivo singular en plural:
def get_plural(singular_noun):
    p = inflect.engine()
    return p.plural(singular_noun)

In [14]:
get_plural('photovoltaic')

'photovoltaics'

In [15]:
# La siguiente función cambiará un sustantivo singular en plural:
def get_singular(plural_noun):
    p = inflect.engine()
    plural = p.singular_noun(plural_noun)
    if (plural):
        return plural
    else:
        return plural_noun

In [17]:
get_singular('equations')

'equation'

Ahora podemos usar las dos funciones anteriores para devolver una lista de sustantivos cambiados en plural o singular, dependiendo del sustantivo original. El siguiente código usa la función is_plural_wn para determinar si el sustantivo es plural. También puede usar la función is_plural_nltk:

In [18]:
def plurals_wn(words_with_pos):
    other_nouns = []
    for noun_info in words_with_pos:
        word = noun_info[0]
        plural = is_plural_wn(word)
        if (plural):
            singular = get_singular(word)
            other_nouns.append(singular)
        else:
            plural = get_plural(word)
            other_nouns.append(plural)
    
    return other_nouns

In [19]:
# Utilice la función anterior para devolver una lista de sustantivos modificados:
other_nouns_wn = plurals_wn(nouns)
print(other_nouns_wn)

['arrays', 'simulations', 'models', 'environments', 'papers', 'models', 'circuits', 'equation', 'photovoltaics', 'cell', 'effect', 'irradiations', 'temperatures', 'change', 'models', 'dcs', 'loads', 'loads', 'inverters', 'validations', 'study', 'loads', 'circuit', 'result']


## Obteniendo análisis de dependencia.

In [62]:
# Importamos librería
import spacy


'''
Recurrir al reporte de errores
'''
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [63]:
# Cargando la frase que se va a analizar
sentence = 'I have seldom heard him mention her under any other name.'

In [64]:
# Cargando el motor spacy
nlp = spacy.load('en_core_web_sm')

In [65]:
# Procesesando la oración con el motor spacy
doc = nlp(sentence)

La información de dependencias estará contenida en el objeto doc. Podemos ver las etiquetas de dependencias haciendo un bucle a través de los tokens en doc:

In [66]:
for token in doc:
    print(token.text, "\t", token.dep_, "\t", spacy.explain(token.dep_))

I 	 nsubj 	 nominal subject
have 	 aux 	 auxiliary
seldom 	 advmod 	 adverbial modifier
heard 	 ROOT 	 None
him 	 nsubj 	 nominal subject
mention 	 ccomp 	 clausal complement
her 	 dobj 	 direct object
under 	 prep 	 prepositional modifier
any 	 det 	 determiner
other 	 amod 	 adjectival modifier
name 	 pobj 	 object of preposition
. 	 punct 	 punctuation


Para explorar la estructura de análisis de dependencias, podemos usar los atributos de la clase Token. Usando sus atributos **ancestors** e **children**, podemos obtener los tokens de los que depende este token y los tokens que dependen de él, respectivamente.

In [67]:
for token in doc:
    print(token.text)
    ancestors = [t.text for t in token.ancestors]
    print(ancestors)

I
['heard']
have
['heard']
seldom
['heard']
heard
[]
him
['mention', 'heard']
mention
['heard']
her
['mention', 'heard']
under
['mention', 'heard']
any
['name', 'under', 'mention', 'heard']
other
['name', 'under', 'mention', 'heard']
name
['under', 'mention', 'heard']
.
['heard']


In [69]:
# tokens secundarios
for token in doc:
    print(token.text)
    children = [t.text for t in token.children]
    print(children)

I
[]
have
[]
seldom
[]
heard
['I', 'have', 'seldom', 'mention', '.']
him
[]
mention
['him', 'her', 'under']
her
[]
under
['name']
any
[]
other
[]
name
['any', 'other']
.
[]


In [70]:
# También podemos ver el subárbol en el que está el token:
for token in doc:
    print(token.text)
    subtree = [t.text for t in token.subtree]
    print(subtree)

I
['I']
have
['have']
seldom
['seldom']
heard
['I', 'have', 'seldom', 'heard', 'him', 'mention', 'her', 'under', 'any', 'other', 'name', '.']
him
['him']
mention
['him', 'mention', 'her', 'under', 'any', 'other', 'name']
her
['her']
under
['under', 'any', 'other', 'name']
any
['any']
other
['other']
name
['any', 'other', 'name']
.
['.']


**Probemos con español**

In [42]:
'''
python -m spacy download es_core_news_sm
'''
nlp = spacy.load("es_core_news_sm")

In [57]:
sentence = 'Me gusta la programación y el mundo maker.'

In [58]:
doc = nlp(sentence)
doc

Me gusta la programación y el mundo maker.

In [59]:
for token in doc:
    print(token.text, "\t", token.dep_, "\t", spacy.explain(token.dep_))

Me 	 obj 	 object
gusta 	 ROOT 	 None
la 	 det 	 determiner
programación 	 nsubj 	 nominal subject
y 	 cc 	 coordinating conjunction
el 	 det 	 determiner
mundo 	 conj 	 conjunct
maker 	 flat 	 flat multiword expression
. 	 punct 	 punctuation


In [60]:
for token in doc:
    print(token.text)
    ancestors = [t.text for t in token.ancestors]
    print(ancestors)

Me
['gusta']
gusta
[]
la
['programación', 'gusta']
programación
['gusta']
y
['mundo', 'programación', 'gusta']
el
['mundo', 'programación', 'gusta']
mundo
['programación', 'gusta']
maker
['mundo', 'programación', 'gusta']
.
['gusta']


## Dividir la frase en cláusulas